In [4]:
# 导入库
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

BiGRU（双向门控循环单元）是一种改进的循环神经网络（RNN）结构，它由两个独立的GRU层组成，一个沿正向处理序列，另一个沿反向处理序列。这种双向结构使得BiGRU能够捕捉到序列中的长距离依赖关系，从而提高模型的性能。

GRU（门控循环单元）是一种RNN变体，它通过引入更新门和重置门来解决传统RNN中的梯度消失问题。更新门负责确定何时更新隐藏状态，而重置门负责确定何时允许过去的信息影响当前隐藏状态。

In [5]:
# 定义BiGRU模型
class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# 生成数据样例
# 均值为1的正态分布随机数
data_0 = np.random.randn(50, 20, 1) + 1
# 均值为-1的正态分布随机数
data_1 = np.random.randn(50, 20, 1) - 1

In [6]:
data_0.shape

(50, 20, 1)

In [7]:
# 合并为总数据集
data = np.concatenate([data_0, data_1], axis=0)
# 将 labels 修改为对应大小的数组
labels = np.concatenate([np.zeros((50, 1)), np.ones((50, 1))], axis=0)

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

# 定义训练和验证函数
def train(model, device, X_train, y_train, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    output = model(X_train.to(device))
    loss = criterion(output, y_train.squeeze().to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

def validate(model, device, X_val, y_val, criterion):
    model.eval()
    with torch.no_grad():
        output = model(X_val.to(device))
        loss = criterion(output, y_val.squeeze().to(device))
    return loss.item()

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 1
hidden_size = 32
num_layers = 1
num_classes = 2
num_epochs = 10
learning_rate = 0.01

model = BiGRU(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_loss = train(model, device, X_train, y_train, optimizer, criterion)
    val_loss = validate(model, device, X_val, y_val, criterion)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# 测试模型
def test(model, device, X_test, y_test):
    model.eval()
    with torch.no_grad():
        output = model(X_test.to(device))
        _, predicted = torch.max(output.data, 1)
        correct = (predicted == y_test.squeeze().to(device)).sum().item()
        accuracy = correct / y_test.size(0)
    return accuracy

test_accuracy = test(model, device, X_val, y_val)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch [1/10], Train Loss: 0.7353, Validation Loss: 0.6560
Epoch [2/10], Train Loss: 0.6422, Validation Loss: 0.5884
Epoch [3/10], Train Loss: 0.5605, Validation Loss: 0.5186
Epoch [4/10], Train Loss: 0.4841, Validation Loss: 0.4433
Epoch [5/10], Train Loss: 0.4079, Validation Loss: 0.3616
Epoch [6/10], Train Loss: 0.3292, Validation Loss: 0.2744
Epoch [7/10], Train Loss: 0.2481, Validation Loss: 0.1852
Epoch [8/10], Train Loss: 0.1680, Validation Loss: 0.1057
Epoch [9/10], Train Loss: 0.0990, Validation Loss: 0.0519
Epoch [10/10], Train Loss: 0.0521, Validation Loss: 0.0248
Test Accuracy: 100.00%
